In [24]:
import matplotlib.pyplot as plt
import torch
import pytorch_measure as pm
import numpy as np
N = 5 # number of atoms
M = 1000 # Number of datapoints
verbose = True
dev = 'cpu'

torch.manual_seed(1)

def regression_model(a,x):
    return a+x

x = torch.linspace(0, 10, M)
data = regression_model(torch.randn(M).to(dev) - 2, x)
#data = -2 + x.view(-1,1) + torch.from_numpy(np.random.normal(0,1,(M,1))).float() # The way of the Sergei

w = torch.rand(N,dtype=torch.float).to(dev)
w = torch.nn.parameter.Parameter(w/w.sum())
l = torch.linspace(-5, 1, N, requires_grad=False).to(dev)

index = []
for i in range(M):
    ab = (regression_model(l, x[i]) - data[i]).abs()
    index.append(torch.argmin(ab))

def NLLLoss(m:list[pm.Measure]):
    return -(m[0].weights[index]).log().sum()

#sd = (l[index] - data)**2
def WardLoss(w):
    return sum(sd * w[index])

def K(d):
        return 1/np.sqrt(2*np.pi)*np.exp(-d**2/2)
h=1.06*M**(-1/5)
# K( (y - yj) / h )
kde_mat = K((data.view(-1,1) - regression_model(l, x.view(-1,1))) / h)

#a = torch.sum(kde_mat, dim=0)

def KDENLLLoss(m):
    return -(torch.matmul(kde_mat, m[0].weights.view(-1,1)) / (M*h)).log().sum()


KeyboardInterrupt: 

In [4]:
measure = pm.Measure(locations=l, weights=w, device=dev)
opt = pm.Optimizer([measure], lr=1e-1)
opt.minimize(KDENLLLoss, verbose=True, print_freq=1, max_epochs=1000, tol_const=1e-2)

plt.plot()

mu=0 #Create true values
sigma=1
xs = l.detach()
y=1/(np.sqrt(2*np.pi)*sigma)*torch.exp(-(xs+2-mu)**2/(2*sigma**2))
y/=sum(y) #Normalize


measure.visualize()
#plt.bar(l-0.1, torch.sum(kde_mat, dim=0)/torch.sum(kde_mat), zorder=0, width=0.1)
plt.scatter(xs, y, zorder=2, label="True distribution")
plt.legend()

plt.show()
#LR         to 1e-4             to 1e-5
#1e-3  -  2147.958496094        2145.816650391
#1e-2  -  2147.958496094        2145.816650391
#1e-1  -  2149.463867188        2145.826660156

Epoch: 0          Loss: 9536.010742188 LR: 0.100000000
Epoch: 1          Loss: 9337.510742188 LR: 0.100000000
Epoch: 2          Loss: 9151.926757812 LR: 0.100000000
Epoch: 3          Loss: 9035.953125000 LR: 0.100000000
Epoch: 4          Loss: 8997.308593750 LR: 0.100000000
Epoch: 5          Lr was reduced to: 0.070000000
Epoch: 6          Loss: 8957.161132812 LR: 0.070000000
Epoch: 7          Loss: 8941.557617188 LR: 0.070000000
Epoch: 8          Lr was reduced to: 0.049000000
Epoch: 9          Lr was reduced to: 0.034300000
Epoch: 10         Lr was reduced to: 0.024010000
Epoch: 11         Loss: 8923.373046875 LR: 0.024010000
Epoch: 12         Lr was reduced to: 0.016807000
Epoch: 13         Loss: 8910.690429688 LR: 0.016807000
Epoch: 14         Lr was reduced to: 0.011764900
Epoch: 15         Lr was reduced to: 0.008235430
Epoch: 16         Loss: 8910.151367188 LR: 0.008235430
Epoch: 17         Lr was reduced to: 0.005764801
Epoch: 18         Loss: 8909.591796875 LR: 0.005764801
Epo

KeyboardInterrupt: 

In [32]:
import sys
sys.path.append('../Sergei')
import torch_measure as s

mes = s.TorchMeasure(l, w)
opt = s.MeasureMinimizer(mes, NLLLoss, learning_rate=1e-1)
opt.minimize()


plt.scatter(xs, y, zorder=2)
plt.bar(l, mes.weights.detach())

TypeError: 'TorchMeasure' object is not subscriptable